<span type="title">访问控制权限</span> | <span type="update">2018-07-18</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍Java的访问控制权限。</p></span>

# 权限控制问题

在面向对象语言中，一个较大的问题是类方法的权限问题，如果对一个对象而言，其所有的域和方法都面向使用者开放，这势必造成混乱，更为麻烦的事，对于类的维护者而言，其几乎无法重构一个已经发布的类，因为类的任何域或者方法都有可能被其余程序调用。Python选择了一种“大家都是成年人”的方式，完全放弃了权限控制，这很糟糕。而在高级的Python编程专著中，往往会有一部分章节用来介绍如何将权限控制加回去的。

Java面临的第一个问题是变与不变的问题，对于用户的接口，也就是API，不变，使用 `public` 关键字声明方法或者类，对于内部的接口，使用 `private protected` 来进行其余的分配。这样就可以在不影响接口的情况下大刀阔斧的修改类，进行重构了。

Java面临的第二个问题是，如何区分Array这个类，如果系统实现了它，而用户又实现了一遍？对于Python而言，其默认用户声明的具有较高的优先级（本地目录），然后查找系统自带的库。对于C而言，使用 `include <stdio.h> 或者 include "stdio.h"` 来进行类似的优先级判定。对于Java而言，则是使用 `package 和 import` 进行管理。

# 文件、构件和群组

`.java` 文件包含Java源代码，称之为一个编译单元。

对于每一个 `.java` 的文件，当使用 `javac` 工具时，会在同目录生成一个 `.class` 的 JVM 二进制字节码，这两个后缀为 `.java, .class` 的文件合起来称之为一个类的构件。

对于一个构件而言，更改只需要发生在 `.java` 文件即可，当系统调用时，会自动进行 `.class` 文件的更新和处理，不用手动进行。

对于 `.java` 源文件而言，其一般包含一个 `public class ClassName` 的类，这个文件的名称也必须为 `ClassName.java`。并且在这个文件中，不能出现两个 `public class ` 类，虽然可以出现多个 `class` 类，其中这些其余的类具有包访问权限，供主要的类进行调用和处理，在包外不可调用。

当把多个 `.java .class` 类构件放在一个文件夹下，就称之为一个群组。

如何告诉Java编译器这些Class文件是一个包呢？对于Python而言，一般需要创建一个空文件夹声明这是一个包，对于Java而言，则必须对每个 `.java` 源文件的**首个非注释行**声明 `package com.mazhangjing.util` 这样一句话，告诉Java这个构件属于 `com.mazhangjing.util` 这个包。也可选择将这个群组打包压缩成 `.jar` 文件，然后将这个文件添加到 `CLASSPATH` 变量中使用。

对于 `com.mzhangjing.util` 包而言，其不是说存在这样一个文件夹叫做 `com.mazhangjing.util`，Java会将句号转换为斜杠，因此这其实是一个三级文件夹，所有的构件都存放在 `com/mazhangjing/util` 这个文件夹下。此外，为了让Java找到这个包，在 `CLASSPATH` 变量中添加 `com` 文件夹存在的父文件夹。

注意，不像Python之类语言会从当前目录开始查找包并自动导入，在 `CLASSPATH` 中如果不存在 `.` 这个目录的话，JAVA是会默认忽略本地文件的。也就是或，当添加 `.` 到变量时，你可以在 `util` 这个目录下执行程序，程序会在 `import util.*` 的时候寻找本地文件夹下的类。

此外，需要注意，对于 `javac` 而言，如果对一个包中的类进行执行的话，如果找不到类，那么添加其包名可能有奇效。`javac com.mazhangjing.util.Print.java` 这样，而不是 `javac Print.java`。因为后者包含 `package com.mazhangjing.com`，在进行编译的时候，它是wu'fa